In [1]:
import os

In [2]:
pwd

'/home/adarsh/DL-E2E/research'

In [3]:
cd ..

/home/adarsh/DL-E2E


In [4]:
pwd

'/home/adarsh/DL-E2E'

In [5]:
import tensorflow as tf

2024-07-16 18:48:21.659400: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-16 18:48:21.714040: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-16 18:48:21.715085: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-16 18:48:23.359489: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class EvaluationConfig:
    path_of_model: Path
    training_data: Path 
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifierr.constants import *
from cnnClassifierr.utils.common import read_yaml ,create_directories ,save_json

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model= 'artifacts/training/model.h5',
            training_data= 'artifacts/data_ingestion/Chicken-fecal-images', 
            all_params= self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE,
        )

        return eval_config

In [12]:
from urllib.parse import urlparse

In [13]:
class Evaluation:
    def __init__(self ,config=EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation='bilinear'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss": self.score[0] ,"accuracy": self.score[1]}
        save_json(path=Path('scores.json') ,data=scores)

In [15]:
try:
    config = ConfigurationManager()
    valid_config = config.get_validation_config()
    evaluation = Evaluation(config=valid_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e

[2024-07-16 19:51:46,416: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-16 19:51:46,420: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-16 19:51:46,422: INFO: common: created directory at: artifacts]
Found 78 images belonging to 2 classes.


2024-07-16 19:51:47.505052: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-07-16 19:51:47.670702: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-07-16 19:51:48.360744: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2024-07-16 19:51:48.654447: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.


1/5 [=====>........................] - ETA: 21s - loss: 0.0126 - accuracy: 1.0000

2024-07-16 19:51:52.449219: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.


5/5 [==============================] - 24s 5s/step - loss: 0.5977 - accuracy: 0.8718
[2024-07-16 19:52:28,405: INFO: common: json file saved at: scores.json]
